In [2]:

import opentrons.simulate as simulate
from opentrons.protocols import bundle
import io
import sys


In [3]:
protocol = io.StringIO()


protocol.write("""
from opentrons import protocol_api

# metadata
metadata = {
    'protocolName': 'My Protocol',
    'author': 'Name <opentrons@example.com>',
    'description': 'Simple protocol to get started using the OT-2',
    'apiLevel': '2.12'
}



# protocol run function
def run(protocol: protocol_api.ProtocolContext):

    # labware
    plate = protocol.load_labware('corning_96_wellplate_360ul_flat', location='1')
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', location='2')

    # pipettes
    left_pipette = protocol.load_instrument(
         'p300_single', mount='left', tip_racks=[tiprack])

    # commands
    left_pipette.pick_up_tip()
    left_pipette.aspirate(100, plate['A1'])
    left_pipette.dispense(100, plate['B2'])
    left_pipette.drop_tip()
""")

with open("my_protocol.py", "w") as f:
    f.write(protocol.getvalue())

In [15]:
sys.argv=[]
"""Run the simulation"""
# parser = argparse.ArgumentParser(
#     prog="opentrons_simulate", description="Simulate an OT-2 protocol"
# )
# parser = simulate.get_arguments(parser)

# args = parser.parse_args()
# Try to migrate api v1 containers if needed

#duration_estimator = DurationEstimator() if args.estimate_duration else None  # type: ignore[no-untyped-call]
duration_estimator = None

with open("my_protocol.py", "r") as protocol:
    runlog, maybe_bundle = simulate.simulate(
        protocol,
        # args.protocol.name,
        # getattr(args, "custom_labware_path", []),
        # getattr(args, "custom_data_path", []) + getattr(args, "custom_data_file", []),
        # duration_estimator=duration_estimator,
        # hardware_simulator_file_path=getattr(args, "custom_hardware_simulator_file"),
        # log_level=args.log_level,
    )

if maybe_bundle:
    bundle_name = "bundle" # getattr(args, "bundle", None)
    # if bundle_name == args.protocol.name:
    #     raise RuntimeError("Bundle path and input path must be different")
    bundle_dest = simulate._get_bundle_dest(
        bundle_name, "PROTOCOL.ot2.zip", "my_protocol" #args.protocol.name
    )
    if bundle_dest:
        bundle.create_bundle(maybe_bundle, bundle_dest)

# if args.output == "runlog":
print(simulate.format_runlog(runlog))

if duration_estimator:
    duration_seconds = duration_estimator.get_total_duration()
    hours = int(duration_seconds / 60 / 60)
    minutes = int((duration_seconds % (60 * 60)) / 60)
    print("--------------------------------------------------------------")
    print(f"Estimated protocol duration: {hours}h:{minutes}m")
    print("--------------------------------------------------------------")
    print("WARNING: Protocol duration estimation is an experimental feature")



Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Aspirating 100.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 150.0 uL/sec
Dispensing 100.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 1 at 300.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12


In [16]:
runlog

[{'level': 0,
  'payload': {'instrument': <InstrumentContext: p300_single_v1 in LEFT>,
   'location': A1 of Opentrons 96 Tip Rack 300 µL on 2,
   'text': 'Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2'},
  'logs': []},
 {'level': 0,
  'payload': {'instrument': <InstrumentContext: p300_single_v1 in LEFT>,
   'volume': 100,
   'location': Location(point=Point(x=14.38, y=74.24, z=4.549999999999999), labware=A1 of Corning 96 Well Plate 360 µL Flat on 1),
   'rate': 1.0,
   'text': 'Aspirating 100.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 150.0 uL/sec'},
  'logs': []},
 {'level': 0,
  'payload': {'instrument': <InstrumentContext: p300_single_v1 in LEFT>,
   'volume': 100,
   'location': Location(point=Point(x=23.38, y=65.24, z=4.549999999999999), labware=B2 of Corning 96 Well Plate 360 µL Flat on 1),
   'rate': 1.0,
   'text': 'Dispensing 100.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 1 at 300.0 uL/sec'},
  'logs': []},
 {'level': 0,
  'payload': {'in